In [ ]:
%matplotlib qt

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
folder = 'sol/'

# Get the files in the folder
import os
files = os.listdir(folder)
# Match files that end in vr.dat
files_vr = [f for f in files if f.endswith('vr.dat')]
files_vr.sort()
files_sol = [f for f in files if f.endswith('solution.dat')]
files_sol.sort()

In [ ]:
ALL_XS = []
ALL_YS = []
ALL_ZS = []
ALL_UXS = []
ALL_UYS = []
ALL_UZS = []
ALL_QXS = []
ALL_QYS = []
ALL_QZS = []


for f in files_vr:
    XS = []
    YS = []
    ZS = []
    UXS = []
    UYS = []
    UZS = []
    QXS = []
    QYS = []
    QZS = []
    with open(folder + f) as file:
        lines = file.readlines()
        vars = lines[0].split()
        
        # From the file name get the NTIME
        ntime = int(f[:5])

        for l in lines[1:]:
            l = l.split()
            try:
                id = int(l[0])
                x = float(l[1])
                y = float(l[2])
                z = float(l[3])
                u = float(l[4])
                v = float(l[5])
                w = float(l[6])
                vort_x = float(l[7])
                vort_y = float(l[8])
                vort_z = float(l[9])
            except:
                print('Error in line: ', l)
                continue

            XS.append(x)
            YS.append(y)
            ZS.append(z)
            UXS.append(u)
            UYS.append(v)
            UZS.append(w)
            QXS.append(vort_x)
            QYS.append(vort_y)
            QZS.append(vort_z)

    XS = np.array(XS)
    YS = np.array(YS)
    ZS = np.array(ZS)
    UXS = np.array(UXS)
    UYS = np.array(UYS)
    UZS = np.array(UZS)
    QXS = np.array(QXS)
    QYS = np.array(QYS)
    QZS = np.array(QZS)

    ALL_XS.append(XS)
    ALL_YS.append(YS)
    ALL_ZS.append(ZS)
    ALL_UXS.append(UXS)
    ALL_UYS.append(UYS)
    ALL_UZS.append(UZS)
    ALL_QXS.append(QXS)
    ALL_QYS.append(QYS)
    ALL_QZS.append(QZS)

In [ ]:
ALL_PM_XS = []
ALL_PM_YS = []
ALL_PM_ZS = []
ALL_PM_UXS = []
ALL_PM_UYS = []
ALL_PM_UZS = []
ALL_PM_QXS = []
ALL_PM_QYS = []
ALL_PM_QZS = []


for f in files_sol:
    XS = []
    YS = []
    ZS = []
    UXS = []
    UYS = []
    UZS = []
    QXS = []
    QYS = []
    QZS = []
    with open(folder + f) as file:
        lines = file.readlines()
        vars = lines[0].split()
        
        # From the file name get the NTIME
        ntime = int(f[:5])
        sizes = [i for i in lines[1].split()]
        IS = int(sizes[1][2:])
        JS = int(sizes[2][2:])
        KS = int(sizes[3][2:])

        for l in lines[3:]:
            l = l.split()
            try:
                x = float(l[0])
                y = float(l[1])
                z = float(l[2])
                u = float(l[3])
                v = float(l[4])
                w = float(l[5])
                vort_x = float(l[6])
                vort_y = float(l[7])
                vort_z = float(l[8])
            except:
                print('Error in line: ', l)
                continue

            XS.append(x)
            YS.append(y)
            ZS.append(z)
            UXS.append(u)
            UYS.append(v)
            UZS.append(w)
            QXS.append(vort_x)
            QYS.append(vort_y)
            QZS.append(vort_z)

    # Convert from Fortran to C
    try:
        XS = np.asfortranarray(XS).reshape(KS, JS, IS, order='C')
        YS = np.asfortranarray(YS).reshape(KS, JS, IS, order='C')
        ZS = np.asfortranarray(ZS).reshape(KS, JS, IS, order='C')
        UXS = np.asfortranarray(UXS).reshape(KS, JS, IS, order='C')
        UYS = np.asfortranarray(UYS).reshape(KS, JS, IS, order='C')
        UZS = np.asfortranarray(UZS).reshape(KS, JS, IS, order='C')
        QXS = np.asfortranarray(QXS).reshape(KS, JS, IS, order='C')
        QYS = np.asfortranarray(QYS).reshape(KS, JS, IS, order='C')
        QZS = np.asfortranarray(QZS).reshape(KS, JS, IS, order='C')
    except:
        print('Error in file: ', f)
    ALL_PM_XS.append(XS)
    ALL_PM_YS.append(YS)
    ALL_PM_ZS.append(ZS)
    ALL_PM_UXS.append(UXS)
    ALL_PM_UYS.append(UYS)
    ALL_PM_UZS.append(UZS)
    ALL_PM_QXS.append(QXS)
    ALL_PM_QYS.append(QYS)
    ALL_PM_QZS.append(QZS)

In [ ]:
print(len(ALL_XS))
print(len(ALL_PM_XS))
print(ALL_PM_XS[0].shape)
x_min = np.min([np.min(x) for x in ALL_XS])
x_max = np.max([np.max(x) for x in ALL_XS])
y_min = np.min([np.min(y) for y in ALL_YS])
y_max = np.max([np.max(y) for y in ALL_YS])
z_min = np.min([np.min(z) for z in ALL_ZS])
z_max = np.max([np.max(z) for z in ALL_ZS])
print(x_min, x_max)
print(y_min, y_max)
print(z_min, z_max)

In [ ]:
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

total_frames = len(ALL_XS)

# Create the figure
# ax: 3D axis for particles
# ax2: 3D axis for mesh points
fig = plt.figure(figsize=(10, 10))
ax_particles: Axes3D = fig.add_subplot(121, projection='3d')
ax_grid_pm: Axes3D = fig.add_subplot(122, projection='3d')
# ax3: 2D axis for vorticity 1
# ax4: 2D axis for vorticity 2
# ax_vorticity1:Axes = fig.add_subplot(223)
# ax_vorticity2:Axes = fig.add_subplot(224)

# Set labels
# Set views
for ax in [ax_particles, ax_grid_pm]:
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=30, azim=30)
    ax.set_aspect('equal')
    ax.set_xlim(x_min, x_max)

# Scatter plots for particles and mesh points
particle_velocities = np.sqrt(ALL_UXS[0]**2 + ALL_UYS[0]**2 + ALL_UZS[0]**2)
particles = ax_particles.scatter(ALL_XS[0], ALL_YS[0], ALL_ZS[0], 
                                c=particle_velocities, s=10, marker='o')
# Colorbar
fig.colorbar(particles, ax=ax_particles)

mesh_velocities = np.sqrt(ALL_PM_UXS[0]**2 + ALL_PM_UYS[0]**2 + ALL_PM_UZS[0]**2)
mesh_points = ax_grid_pm.scatter(ALL_PM_XS[0], ALL_PM_YS[0], ALL_PM_ZS[0],
                                c=mesh_velocities, s=0.5, marker='x')
# Colorbar
fig.colorbar(mesh_points, ax=ax_grid_pm)

slice_1 = np.s_[:,:,np.shape(ALL_PM_YS[0])[2]//2]
slice_2 = np.s_[:,np.shape(ALL_PM_YS[0])[1]//2,:]

# Set labels for particle plots
ax_particles.set_title('Particles')
ax_particles.set_xlabel('X')
ax_particles.set_ylabel('Y')
ax_particles.set_zlabel('Z')

ax_grid_pm.set_title('Mesh Points')
ax_grid_pm.set_xlabel('X')
ax_grid_pm.set_ylabel('Y')
ax_grid_pm.set_zlabel('Z')

# vorticity1 = np.sqrt(ALL_PM_QXS[0]**2 + ALL_PM_QYS[0]**2 + ALL_PM_QZS[0]**2)
# vorticity2 = np.sqrt(ALL_PM_QXS[0]**2 + ALL_PM_QYS[0]**2 + ALL_PM_QZS[0]**2)
# # Initial contours
# contour1 = ax_vorticity1.contourf(ALL_PM_XS[0][slice_1], ALL_PM_YS[0][slice_1], vorticity1[slice_1], cmap='viridis')
# fig.colorbar(contour1, ax=ax_vorticity1)

# contour2 = ax_vorticity2.contourf(ALL_PM_XS[0][slice_2], ALL_PM_ZS[0][slice_2], vorticity2[slice_2], cmap='viridis')
# fig.colorbar(contour2, ax=ax_vorticity2)
# # Set labels for vorticity plots
# ax_vorticity1.set_title('Vorticity 1 (X-Y plane) Z=0')
# ax_vorticity1.set_xlabel('X')
# ax_vorticity1.set_ylabel('Y')

# ax_vorticity2.set_title('Vorticity 2 (X-Z plane) Y=0')
# ax_vorticity2.set_xlabel('X')
# ax_vorticity2.set_ylabel('Z')

def update(frame):
    # Non local variables
    global contour1, contour2
    # Update particle positions
    print(f"Frame {frame} / {total_frames}")
    particle_velocities = np.sqrt(ALL_UXS[frame]**2 + ALL_UYS[frame]**2 + ALL_UZS[frame]**2)
    # Keep only the 10% of the strongest velocities
    XPR = ALL_XS[frame]
    YPR = ALL_YS[frame]
    ZPR = ALL_ZS[frame]
    UPR = particle_velocities
    
    particles.set_offsets(np.c_[XPR, YPR])
    particles.set_3d_properties(ZPR, 'z')
    particles.set_array(UPR)
    # Reset the color limits and autoscale
    particles.set_clim(particle_velocities.min(), particle_velocities.max())
    ax_particles.autoscale_view()

    # Update mesh point positions
    mesh_velocities = np.sqrt(ALL_PM_UXS[frame]**2 + ALL_PM_UYS[frame]**2 + ALL_PM_UZS[frame]**2)
    mesh_points.set_offsets(np.c_[ALL_PM_XS[frame].ravel(), ALL_PM_YS[frame].ravel()])
    mesh_points.set_3d_properties(ALL_PM_ZS[frame].ravel(), 'z')
    # Calculate magnitudes for coloring mesh points
    mesh_points.set_array(mesh_velocities.ravel())
    # Reset the color and view limits
    mesh_points.set_clim(mesh_velocities.min(), mesh_velocities.max())
    ax_grid_pm.autoscale_view()

    # Update vorticity
    vorticity1 = np.sqrt(ALL_PM_QXS[frame]**2 + ALL_PM_QYS[frame]**2 + ALL_PM_QZS[frame]**2)
    vorticity2 = np.sqrt(ALL_PM_QXS[frame]**2 + ALL_PM_QYS[frame]**2 + ALL_PM_QZS[frame]**2)
    print(f"\tMin {np.min(vorticity1[slice_1])} Max {np.max(vorticity1[slice_1])} ")
    print(f"\tMin {np.min(vorticity2[slice_2])} Max {np.max(vorticity2[slice_2])} ")

    # Update contours
    # for c in contour1.collections:
    #     c.remove()
    # for c in contour2.collections:
    #     c.remove()
    
    # contour1 = ax_vorticity1.contourf(ALL_PM_XS[frame][slice_1], ALL_PM_YS[frame][slice_1], vorticity1[slice_1], cmap='viridis')
    # contour2 = ax_vorticity2.contourf(ALL_PM_XS[frame][slice_2], ALL_PM_ZS[frame][slice_2], vorticity2[slice_2], cmap='viridis')

    # Update colorbars
    # contour1.changed()
    # contour2.changed()

    # Update figure title
    fig.suptitle(f"Frame {frame} / {99}")
    # fig.canvas.draw()

    return particles, mesh_points #, contour1, contour2


ani = FuncAnimation(fig, update, frames=total_frames-1, blit=False, repeat=False, interval=100)
fig.canvas.draw()
fig.tight_layout()
ani.event_source.stop()
ani.save(f"{folder}/anim.gif")